## 연습문제

단한 은행 컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.

은행 컨트랙의 함수는 다음과 같이 만드세요.

* 입금액을 정해서 입금. 입금액을 매개변수로 넣어준다.

	function deposit(uint amount) public payable

* 전액 인출 함수. 인출 금액을 정하기 않아도 되므로 함수 인자가 없다.

	function widthdrawAll() public

* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)

	function getBalance() public view returns(uint, uint)

* 다른 계정으로 계좌이체 함수

	function forwardTo(address payable _receiver) public payable



컨트랙을 ganache에 배포하고, 아래 기능을 실행해 보세요.

- 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)



### ganache

### 1. 개발

In [52]:
%%writefile ../src/MyBank.sol
pragma solidity 0.6.4;

contract MyBank {
    address payable owner;
    uint balance;
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }
    
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public {
        uint amount = balance;
        balance -= amount;
        owner.transfer(amount);
    }
    function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balance);
    }
    function forwardTo(address payable _receiver) public payable {
        uint amount = 333;
        balance -= amount;
        _receiver.transfer(amount);
    }
}

Overwriting ../src/MyBank.sol


### 2. 컴파일

In [53]:
!solc --abi --bin --gas ../src/MyBank.sol


======= ../src/MyBank.sol:MyBank =======
Gas estimation:
construction:
   41047 + 116600 = 157647
external:
   deposit(uint256):	21103
   forwardTo(address):	infinite
   getBalance():	1027
   withdrawAll():	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610247806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad8814610076578063853828b6146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610170565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101f2565b005b60008047600154915091509091565b600061014d90508060016000828

### 3. 배포

In [54]:
%%writefile ../src/MyBankDeploy.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610247806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad8814610076578063853828b6146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610170565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101f2565b005b60008047600154915091509091565b600061014d9050806001600082825403925050819055508173ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f1935050505015801561016b573d6000803e3d6000fd5b505050565b60006001549050806001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f193505050501580156101ee573d6000803e3d6000fd5b5050565b8034146101fe57600080fd5b806001600082825401925050819055505056fea2646970667358221220cda7fc4d9e8dc978a01a0817cee5ef25976afcca4cf010d9faab727e3186e1ff64736f6c63430006040033";
var _contract = new web3.eth.Contract(_abi);

_contract
    .deploy({ data: "0x"+_bin })
    .send({
     from: '0xF4B615BDCDbfB8cE347cf2581761cBb0Ce08da34',
     gas: '1000000'
    })
    .then(function(newContractInstance){
        console.log("contract address: "+newContractInstance.options.address)
    });

Overwriting ../src/MyBankDeploy.js


In [55]:
!node ../src/MyBankDeploy.js

contract address: 0xf8F8aefa8Fb920612750686E4c8F263bFb36138D


### 4. 사용

### 1) 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

In [56]:
%%writefile ../src/MyBankUse1.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xf8F8aefa8Fb920612750686E4c8F263bFb36138D");
var coinbase = "0xF4B615BDCDbfB8cE347cf2581761cBb0Ce08da34";

bank.methods.deposit(11111).send({from:coinbase, gas:1000000, value:11111});
bank.methods.deposit(222).send({from:coinbase, gas:1000000, value:222});

Overwriting ../src/MyBankUse1.js


In [60]:
%%writefile ../src/MyBankUse-chk-contract.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xf8F8aefa8Fb920612750686E4c8F263bFb36138D");
var coinbase = "0xF4B615BDCDbfB8cE347cf2581761cBb0Ce08da34";

bank.methods.getBalance().call().then(console.log);

Overwriting ../src/MyBankUse-chk-contract.js


In [58]:
!node ../src/MyBankUse1.js

In [62]:
!node ../src/MyBankUse-chk-contract.js

Result { '0': '11333', '1': '11333' }


### 2) 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

In [65]:
%%writefile ../src/MyBankUse2-chk.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xf8F8aefa8Fb920612750686E4c8F263bFb36138D");
var coinbase = "0xF4B615BDCDbfB8cE347cf2581761cBb0Ce08da34";
var receiver = "0x3EcAf48D1C831fB1029d23c23728c09061b79A62";

web3.eth.getBalance(receiver).then(function(ret) {
    console.log("receiver balance: "+ret);
})

Overwriting ../src/MyBankUse2-chk.js


In [66]:
!node ../src/MyBankUse2-chk.js

receiver balance: 100000000000000000000


In [74]:
%%writefile ../src/MyBankUse2.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xf8F8aefa8Fb920612750686E4c8F263bFb36138D");
var coinbase = "0xF4B615BDCDbfB8cE347cf2581761cBb0Ce08da34";
var receiver = "0x3EcAf48D1C831fB1029d23c23728c09061b79A62";

bank.methods.forwardTo(receiver).send({from:coinbase, gas:1000000});

Overwriting ../src/MyBankUse2.js


In [68]:
!node ../src/MyBankUse2.js

In [69]:
!node ../src/MyBankUse2-chk.js

receiver balance: 100000000000000000333


In [70]:
!node ../src/MyBankUse-chk-contract.js

Result { '0': '11000', '1': '11000' }


-> receiver balance가 333 wei만큼 증가하였으며, 컨트랙 잔고가 11000 wei로 설정되었다.

### 3) 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

In [71]:
%%writefile ../src/MyBankUse3-chk.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xf8F8aefa8Fb920612750686E4c8F263bFb36138D");
var coinbase = "0xF4B615BDCDbfB8cE347cf2581761cBb0Ce08da34";

web3.eth.getBalance(coinbase).then(function(ret) {
    console.log("coinbase balance: "+ret);
})

Writing ../src/MyBankUse3-chk.js


In [72]:
!node ../src/MyBankUse3-chk.js

coinbase balance: 99970624519999932002


In [32]:
%%writefile ../src/MyBankUse3.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xf8F8aefa8Fb920612750686E4c8F263bFb36138D");
var coinbase = "0xF4B615BDCDbfB8cE347cf2581761cBb0Ce08da34";

bank.methods.withdrawAll().send({from:coinbase, gas:1000000});

Overwriting ../src/MyBankUse3.js


In [76]:
!node ../src/MyBankUse3.js

In [77]:
!node ../src/MyBankUse3-chk.js

coinbase balance: 99970200199999943002


In [78]:
!node ../src/MyBankUse-chk-contract.js

Result { '0': '0', '1': '0' }


-> coinbase balance가 뒤에 5자리만 봤을 때 11000 wei만큼 증가하였으며, 컨트랙 잔고가 0 wei로 설정되었다.

---
### geth@8445

pragma solidity 0.5.0으로 REMIX에서 컴파일 후 결과를 사용했습니다.

### 3. 배포

In [1]:
!geth --exec "eth.coinbase" attach http://127.0.0.1:8445

"0x74ccd5b995b3480fe9063ee539ebf065385f61c8"


In [16]:
%%writefile ../src/MyBankDeploy-geth.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}

var _abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin = "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055503073ffffffffffffffffffffffffffffffffffffffff16316001819055506102798061007e6000396000f3fe608060405260043610610062576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe01461006757806327d8ad8814610099578063853828b6146100dd578063b6b55f25146100f4575b600080fd5b34801561007357600080fd5b5061007c610122565b604051808381526020018281526020019250505060405180910390f35b6100db600480360360208110156100af57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610148565b005b3480156100e957600080fd5b506100f26101aa565b005b6101206004803603602081101561010a57600080fd5b810190808035906020019092919050505061022c565b005b6000803073ffffffffffffffffffffffffffffffffffffffff1631600154915091509091565b600061014d9050806001600082825403925050819055508173ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f193505050501580156101a5573d6000803e3d6000fd5b505050565b60006001549050806001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f19350505050158015610228573d6000803e3d6000fd5b5050565b803414151561023a57600080fd5b806001600082825401925050819055505056fea165627a7a7230582089ef82d58762171f4c91db27996cdc11a23944ad0b9f7b3937119f7e9d4961010029";
var _contract = new web3.eth.Contract(_abi);

_contract
    .deploy({ data: "0x"+_bin })
    .send({
     from: '0x74ccd5b995b3480fe9063ee539ebf065385f61c8',
     gas: '4700000'
    })
    .then(function(newContractInstance){
        console.log("contract address: "+newContractInstance.options.address)
    });

Overwriting ../src/MyBankDeploy-geth.js


In [18]:
!node ../src/MyBankDeploy-geth.js

contract address: 0x4BA8054752dC7c0823DdcA16a783E15eB9edf3eD


### 4. 사용

### 1) 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

In [19]:
%%writefile ../src/MyBankUse1-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var bank = new web3.eth.Contract(_abi, "0x4BA8054752dC7c0823DdcA16a783E15eB9edf3eD");
var coinbase = "0x74ccd5b995b3480fe9063ee539ebf065385f61c8";

bank.methods.deposit(11111).send({from:coinbase, gas:1000000, value:11111});
bank.methods.deposit(222).send({from:coinbase, gas:1000000, value:222});

Overwriting ../src/MyBankUse1-geth.js


In [20]:
%%writefile ../src/MyBankUse-chk-contract-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var bank = new web3.eth.Contract(_abi, "0x4BA8054752dC7c0823DdcA16a783E15eB9edf3eD");
var coinbase = "0x74ccd5b995b3480fe9063ee539ebf065385f61c8";

bank.methods.getBalance().call().then(console.log);

Overwriting ../src/MyBankUse-chk-contract-geth.js


In [21]:
!node ../src/MyBankUse1-geth.js

In [22]:
!node ../src/MyBankUse-chk-contract-geth.js

Result { '0': '11333', '1': '11333' }


### 2) 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

In [11]:
!geth --exec "eth.accounts[1]" attach http://127.0.0.1:8445

"0x3c4fba0dde9ea0f44b41321dcfc49e134f63cfa2"


In [23]:
%%writefile ../src/MyBankUse2-chk-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var bank = new web3.eth.Contract(_abi, "0x4BA8054752dC7c0823DdcA16a783E15eB9edf3eD");
var coinbase = "0x74ccd5b995b3480fe9063ee539ebf065385f61c8";
var receiver = "0x3c4fba0dde9ea0f44b41321dcfc49e134f63cfa2";

web3.eth.getBalance(receiver).then(function(ret) {
    console.log("receiver balance: "+ret);
})

Overwriting ../src/MyBankUse2-chk-geth.js


In [24]:
!node ../src/MyBankUse2-chk-geth.js

receiver balance: 10000000000000010000


In [25]:
%%writefile ../src/MyBankUse2-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var bank = new web3.eth.Contract(_abi, "0x4BA8054752dC7c0823DdcA16a783E15eB9edf3eD");
var coinbase = "0x74ccd5b995b3480fe9063ee539ebf065385f61c8";
var receiver = "0x3c4fba0dde9ea0f44b41321dcfc49e134f63cfa2";

bank.methods.forwardTo(receiver).send({from:coinbase, gas:1000000});

Overwriting ../src/MyBankUse2-geth.js


In [26]:
!node ../src/MyBankUse2-geth.js

In [27]:
!node ../src/MyBankUse2-chk-geth.js

receiver balance: 10000000000000010333


In [28]:
!node ../src/MyBankUse-chk-contract-geth.js

Result { '0': '11000', '1': '11000' }


-> receiver balance가 333 wei만큼 증가하였으며, 컨트랙 잔고가 11000 wei로 설정되었다.

### 3) 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

In [29]:
%%writefile ../src/MyBankUse3-chk-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var bank = new web3.eth.Contract(_abi, "0x4BA8054752dC7c0823DdcA16a783E15eB9edf3eD");
var coinbase = "0x74ccd5b995b3480fe9063ee539ebf065385f61c8";

web3.eth.getBalance(coinbase).then(function(ret) {
    console.log("coinbase balance: "+ret);
})

Writing ../src/MyBankUse3-chk-geth.js


In [30]:
!node ../src/MyBankUse3-chk-geth.js

coinbase balance: 1094531249999999837334


In [33]:
%%writefile ../src/MyBankUse3-geth.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abi = [{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var bank = new web3.eth.Contract(_abi, "0x4BA8054752dC7c0823DdcA16a783E15eB9edf3eD");
var coinbase = "0x74ccd5b995b3480fe9063ee539ebf065385f61c8";

bank.methods.withdrawAll().send({from:coinbase, gas:1000000});

Writing ../src/MyBankUse3-geth.js


In [34]:
!node ../src/MyBankUse3-geth.js

In [35]:
!node ../src/MyBankUse3-chk-geth.js

coinbase balance: 1119531249999999848334


In [36]:
!node ../src/MyBankUse-chk-contract-geth.js

Result { '0': '0', '1': '0' }


-> coinbase balance가 뒤에 5자리만 봤을 때 11000 wei만큼 증가하였으며, 컨트랙 잔고가 0 wei로 설정되었다.